In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

169017344/169001437 [==============================] - 2s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(100))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('swish')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 80s 90ms/step - loss: 4.7081 - accuracy: 0.0257 - top3_acc: 0.0700 - top_k_categorical_accuracy: 0.1062 - val_loss: 4.3436 - val_accuracy: 0.0685 - val_top3_acc: 0.1487 - val_top_k_categorical_accuracy: 0.2056
Epoch 2/50
313/313 [==============================] - 26s 83ms/step - loss: 4.1176 - accuracy: 0.0734 - top3_acc: 0.1748 - top_k_categorical_accuracy: 0.2487 - val_loss: 3.9134 - val_accuracy: 0.1104 - val_top3_acc: 0.2359 - val_top_k_categorical_accuracy: 0.3267
Epoch 3/50
313/313 [==============================] - 26s 84ms/step - loss: 3.8707 - accuracy: 0.1107 - top3_acc: 0.2416 - top_k_categorical_accuracy: 0.3268 - val_loss: 3.6564 - val_accuracy: 0.1658 - val_top3_acc: 0.3245 - val_top_k_categorical_accuracy: 0.4190
Epoch 4/50
313/313 [==============================] - 26s 83ms/step - loss: 3.6977 - accuracy: 0.1405 - top3_acc: 0.2923 - top_k_categorical_accuracy: 0.3849 - val_loss: 3.6084 - val_accuracy: 0.1813 - val_to

In [8]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.501015663146973, 4.05163049697876, 3.8307862281799316, 3.673764944076538, 3.555211305618286, 3.454939365386963, 3.3539133071899414, 3.2672924995422363, 3.175534963607788, 3.0978543758392334, 3.0159993171691895, 2.94769287109375, 2.8718931674957275, 2.804564952850342, 2.7319037914276123, 2.669278621673584, 2.610605239868164, 2.5421602725982666, 2.4842307567596436, 2.425259590148926, 2.370404005050659, 2.3074228763580322, 2.2462804317474365, 2.1877198219299316, 2.1258702278137207, 2.0732622146606445, 2.0241758823394775, 1.9610720872879028, 1.9031386375427246, 1.8416457176208496, 1.7760933637619019, 1.7218371629714966, 1.6625697612762451, 1.6009632349014282, 1.5397469997406006, 1.4857383966445923, 1.4371771812438965, 1.3800990581512451, 1.3189727067947388, 1.2713967561721802, 1.2173657417297363, 1.1621183156967163, 1.1076565980911255, 1.0607869625091553, 1.0039924383163452, 0.9655655026435852, 0.9148514866828918, 0.8595556616783142, 0.824164867401123, 0.7728752493858337], 'acc